In [ ]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import pyarrow.parquet as pq
import os
import json
from io import StringIO

# --- Configuratie ---
# Pad naar je Parquet-bestand met verkeersdata.
TOMTOM_FILE_PATH = os.path.join('Data', '20250820163000_stream.tomtom.analyze-sail.parquet')
# Pad naar het SHP-bestand. AANGEPAST naar het volledige pad dat je hebt opgegeven.
# De 'r' voor de string zorgt ervoor dat de backslashes in het Windows-pad correct worden gelezen.
SHAPEFILE_PATH = r'C:\Users\gerri\Downloads\01-12-2024\01-12-2024\Wegvakken\Wegvakken.shp'
# We nemen een 10% sample van de data om geheugenproblemen te voorkomen.
SAMPLE_FRACTION = 0.1
# Dit is de naam van de kolom in het shapefile die de ID's van de wegvakken bevat.
# 'WVK_ID' is een veelvoorkomende naam in het NWB. Pas dit aan als het anders heet.
SHAPEFILE_ID_COL = 'WVK_ID'


# --- Helper Functie om TomTom Data te Verwerken ---
def parse_tomtom_data(value_string):
    """
    Verwerkt de JSON-string uit de '_value' kolom en de CSV-data daarbinnen.
    """
    try:
        start_index = value_string.find('{')
        if start_index == -1: return None
        data = json.loads(value_string[start_index:])
        
        if 'data' in data and isinstance(data['data'], str):
            csv_string = data['data']
            return pd.read_csv(StringIO(csv_string))
            
    except (json.JSONDecodeError, KeyError):
        return None
    return None

# --- Hoofdscript ---
try:
    # --- 1. Laad en verwerk TomTom Verkeersdata ---
    print(f"Laden van {SAMPLE_FRACTION*100}% sample uit {TOMTOM_FILE_PATH}...")
    parquet_file = pq.ParquetFile(TOMTOM_FILE_PATH)
    list_of_sampled_dfs = []
    for batch in parquet_file.iter_batches():
        chunk_df = batch.to_pandas()
        list_of_sampled_dfs.append(chunk_df.sample(frac=SAMPLE_FRACTION))
    df_raw = pd.concat(list_of_sampled_dfs, ignore_index=True)
    print("Ruwe data succesvol geladen en gesampled!")

    print("Verwerken van verkeersdata uit de '_value' kolom...")
    parsed_dfs = df_raw['_value'].apply(parse_tomtom_data)
    parsed_dfs.dropna(inplace=True)
    df_traffic = pd.concat(parsed_dfs.tolist(), ignore_index=True)
    print(f"Succesvol {len(df_traffic):,} wegsegment-datapunten verwerkt.")
    
    # --- Aggregeer de verkeersdata ---
    print("\nAggregeren van verkeersdata: berekenen van gemiddelde drukte per wegvak...")
    df_traffic_agg = df_traffic.groupby('id')['traffic_level'].mean().reset_index()
    print(f"Data geaggregeerd naar {len(df_traffic_agg):,} unieke wegvakken.")

    # --- 2. Laad de Kaartdata (Shapefile) ---
    print(f"\nLaden van kaartdata uit shapefile: {SHAPEFILE_PATH}...")
    gdf = gpd.read_file(SHAPEFILE_PATH)
    print("Kaartdata succesvol geladen!")
    
    # --- Filter kaartdata op Amsterdam ---
    print("Coördinatensysteem van shapefile omzetten...")
    gdf = gdf.to_crs(epsg=44326) # Eerst omzetten naar standaard lat/lon
    
    print("Filteren van wegvakken binnen de bounding box van Amsterdam...")
    # Definieer de bounding box voor de regio Amsterdam
    min_lon, min_lat = 4.72, 52.28
    max_lon, max_lat = 5.08, 52.43
    gdf_amsterdam = gdf.cx[min_lon:max_lon, min_lat:max_lat]
    print(f"Filteren voltooid: {len(gdf_amsterdam):,} wegvakken over in de regio Amsterdam.")

    
    # --- 3. Koppel de dataframes ---
    if SHAPEFILE_ID_COL not in gdf_amsterdam.columns:
        raise KeyError(f"KRITISCHE FOUT: De ID-kolom '{SHAPEFILE_ID_COL}' is niet gevonden in het shapefile. "
                       f"Pas de 'SHAPEFILE_ID_COL' variabele bovenaan het script aan.")

    print(f"\nKoppelen van verkeersdata aan de gefilterde kaartdata...")
    gdf_amsterdam[SHAPEFILE_ID_COL] = pd.to_numeric(gdf_amsterdam[SHAPEFILE_ID_COL], errors='coerce')
    df_traffic_agg['id'] = pd.to_numeric(df_traffic_agg['id'], errors='coerce')
    
    # We mergen nu met de veel kleinere, gefilterde GeoDataFrame
    merged_gdf = gdf_amsterdam.merge(df_traffic_agg, left_on=SHAPEFILE_ID_COL, right_on='id')
    print(f"Succesvol {len(merged_gdf):,} wegvakken gekoppeld!")

    if len(merged_gdf) == 0:
         print("\nWAARSCHUWING: Er konden geen wegvakken worden gekoppeld. Controleer of de ID-kolommen ('id' en de gekozen SHAPEFILE_ID_COL) overeenkomen.")
    else:
        # --- 4. Creëer de Choropleth Kaart ---
        print("Genereren van de verkeerskaart...")
        
        # AANPASSING: Converteer datumkolommen naar een string voor JSON-compatibiliteit.
        for col in merged_gdf.columns:
            if pd.api.types.is_datetime64_any_dtype(merged_gdf[col]):
                print(f"Datumkolom '{col}' wordt omgezet naar string...")
                merged_gdf[col] = merged_gdf[col].astype(str)

        # Converteer de gefusioneerde data naar een expliciete GeoJSON.
        merged_gdf_json = json.loads(merged_gdf.to_json())

        fig = px.choropleth_mapbox(
            merged_gdf,
            geojson=merged_gdf_json,
            locations=SHAPEFILE_ID_COL,
            featureidkey=f"properties.{SHAPEFILE_ID_COL}",
            color="traffic_level",
            color_continuous_scale="RdYlGn_r",
            mapbox_style="carto-positron",
            center={"lat": 52.3676, "lon": 4.9041},
            zoom=11,
            opacity=0.8,
            title="Gemiddelde Verkeersdrukte per Wegvak in Amsterdam",
            labels={"traffic_level": "Gem. Drukte"},
            height=800
        )
        fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
        fig.show()
        print("\nVisualisatie gegenereerd! Controleer je web browser.")


except FileNotFoundError as e:
    print(f"--- FOUT: Bestand niet gevonden ---")
    print(f"Het script kon een bestand niet vinden: {e.filename}")

except Exception as e:
    print(f"Er is een onverwachte fout opgetreden: {e}")



Laden van 10.0% sample uit Data\20250820163000_stream.tomtom.analyze-sail.parquet...
Ruwe data succesvol geladen en gesampled!
Verwerken van verkeersdata uit de '_value' kolom...
Succesvol 225,582 wegsegment-datapunten verwerkt.

Aggregeren van verkeersdata: berekenen van gemiddelde drukte per wegvak...
Data geaggregeerd naar 10,742 unieke wegvakken.

Laden van kaartdata uit shapefile: C:\Users\gerri\Downloads\01-12-2024\01-12-2024\Wegvakken\Wegvakken.shp...
